In [1]:
pip install tensorflow, keras, pickle, nltk

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'tensorflow,'
You should consider upgrading via the 'C:\Users\snegi\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json', encoding="utf8").read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
classes

['வணக்கம்',
 'விடைபெறுகிறேன்',
 'நன்றி',
 'விருப்பங்கள்',
 'என்.எல்.பி',
 'பயன்பாட்டு வழக்குகள் ']

In [6]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower())
for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

28 documents
6 classes ['என்.எல்.பி', 'நன்றி', 'பயன்பாட்டு வழக்குகள் ', 'வணக்கம்', 'விடைபெறுகிறேன்', 'விருப்பங்கள்']
53 unique lemmatized words [',', 'அடுத்த', 'அது', 'அரட்டையடிப்பது', 'அருமை', 'ஆதரவு', 'இனிய', 'இயற்கை', 'இரவு', 'இருக்கிறது', 'இருக்கிறார்களா', 'இருக்கிறீர்கள்', 'உங்களுடன்', 'உதவ', 'உதவியதற்கு', 'உதவியாக', 'உன்னால்', 'எங்கே', 'எதைக்', 'எனக்கு', 'என்.எல்.பி', 'என்எல்பி', 'என்எல்பியின்', 'என்ன', 'என்பது', 'என்றால்', 'எப்படி', 'ஏய்', 'ஐ', 'காலை', 'குறிக்கிறது', 'செயலாக்கம்', 'செய்ய', 'நண்பரே', 'நன்றி', 'நாள்', 'நீங்கள்', 'பயன்படுத்தலாம்', 'பயன்பாடுகள்', 'பார்க்கலாம்', 'பிறகு', 'மதிய', 'மாலை', 'மிக்க', 'முடியும்', 'முறை', 'மொழி', 'யாராவது', 'வணக்கம்', 'வருகிறேன்', 'வரை', 'வழங்கப்படுகிறது', 'விடைபெறுகிறேன்']


In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
# create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-7-829d2e54a249>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

C:\Users\snegi\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
6/6 [==============================] - 1s 5ms/step - loss: 1.7973 - accuracy: 0.0714
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 1.7474 - accuracy: 0.3929
Epoch 3/200
6/6 [==============================] - 0s 5ms/step - loss: 1.6739 - accuracy: 0.3214
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 1.6438 - accuracy: 0.4286
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.5424 - accuracy: 0.4286
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 1.3769 - accuracy: 0.5357
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 1.4050 - accuracy: 0.5000
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2837 - accuracy: 0.5714
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2075 - accuracy: 0.5714
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 1.1600 - accuracy: 0.6429
Epoch 11/200
6/6 [===========